----
# XGBoost practice
---


### Dataset
We will again use the [pima indian diabetes dataset](pima-indians-diabetes.csv), whose description is [here](http://archive.ics.uci.edu/ml/machine-learning-databases/pima-indians-diabetes/pima-indians-diabetes.names), which records measurements about several hundred patients and an indication of whether or not they tested positive for diabetes (the class label).  The classification is therefore to predict whether a patient will test positive for diabetes, based on the various measurements.

The dataset has been downloaded as a csv file in the current directory. 

## Exercise 0: prelude
Install and import necessary modules and functions including: 
 * pandas for loading and parsing data. 
 * `cross_val_score` from sklearn to do cross validation automatically. 
 * `xgboost` because we are using this model today. 

Load the pima data into a pandas dataframe. Do some exploration to gain some understanding of the dataset. 

In [ ]:
###Answer to 1a) here
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import linear_model
# from sklearn.cross_validation import train_test_split
from sklearn import metrics
# from sklearn.cross_validation import cross_val_score
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn import preprocessing

%matplotlib inline

In [ ]:
##load in the data
data_filepath = '../test_data/pima-indians-diabetes.csv'
pima=pd.read_csv(data_filepath,encoding = 'ISO-8859-1')

##get just the features
data=pima[['numpregnant','plasma','blood pressure','sf-thickness','serum-insulin','BMI','pedigree-function','age']]

##get just the class labels
classlabel=pima['has_diabetes']

In [ ]:
#conda install -c conda-forge py-xgboost
from xgboost import XGBClassifier
xgboostmodel = XGBClassifier(
    n_estimators=100,
    eta=0.3,
    min_child_weight=1,
    max_depth=3, 
    # add the following to remove two depreciation warning
    eval_metric='mlogloss', 
    use_label_encoder=False
)

xgboostmodel.fit(data, classlabel)

## Exercise 1: Define `xgboost` model
Define an `xgboost` model for our data. Since we are doing (binary) classification, we will use `XGBClassifier`. 
Do: 
 * Read the docs:
   * [Get started](https://xgboost.readthedocs.io/en/latest/get_started.html)
   * [XGBClassifier]
 * Take note on the various parameters.
 * Instantiate the classifier. 

### Performance evaluation
a) Report the AUC of an XGBoost classifier on the dataset using 10 fold cross validation.

b) Manually vary the following parameters and observe the effect on AUC of varying the following parameters for XGBoost? 

    • max depth
    • learning rate 
    • n estimators 
    • gamma 
    • min child weight 
    • reg lambda

In [ ]:
print("10-fold cross validation AUC= ",cross_val_score(xgboostmodel, data,classlabel,cv=10,scoring='roc_auc').mean())

## Exercise 2: Effect of hyperparameters
The following example illustrates how one can explore the resulting performance of a classifier a we vary its   hyperparameter (`max_depth` in this case) by plotting `parameter vs performance` on a graph. 


In [ ]:
# Define a range of the hyperparameter. 
max_depths = range(1,15,2)

# a list to accumulate the performance measure. We use accuracy for example. Change it as you wish.
performance_scores=[]

# Define the model. 
xgboostmodel = XGBClassifier(
    n_estimators=100,
    eta=0.3,
    min_child_weight=1,
    max_depth=3, 
    # add the following to remove two depreciation warning
    eval_metric='mlogloss', 
    use_label_encoder=False
)

# In a loop, assign different hyperparameter to the model and record performance measure. 
for m_depth in max_depths:
    # reassign the hyperparameter
    xgboostmodel.max_depth = m_depth
    
    # Compute performance score (we use the mean of 10-fold cross validation with ROCAUC as score each time)
    score = cross_val_score(xgboostmodel, data,classlabel,cv=10,scoring='roc_auc').mean()
    
    # print it out if you wish
    #print("10-fold cross validation AUC= ",cross_val_score(xgboostmodel, data,classlabel,cv=10,scoring='roc_auc').mean())
    performance_scores.append(score)

plt.plot(max_depths, performance_scores)
plt.xlabel('Max-Depth', fontsize=16)
plt.ylabel('AUC', fontsize=16)

#### Booster Parameters
2) eta [default=0.3]

    - Analogous to learning rate in GBM
    - Makes the model more robust by shrinking the weights on each step
    - Typical final values to be used: 0.01-0.2

In [ ]:
import numpy as np
l_rates = np.arange(0.01,0.2,0.01)
accuracies=[]
for l_r in l_rates:
    xgboostmodel.eta = l_r

#     xgboostmodel = XGBClassifier(n_estimators=100,eta=l_r,min_child_weight=1,max_depth=7)

    #print("10-fold cross validation AUC= ",cross_val_score(xgboostmodel, data,classlabel,cv=10,scoring='roc_auc').mean())
        
    accuracies.append(cross_val_score(xgboostmodel, data,classlabel,cv=10,scoring='roc_auc').mean())

plt.plot(l_rates,accuracies)
plt.xlabel('Learning Rate', fontsize=16)
plt.ylabel('Accuracy', fontsize=16)


#### Booster Parameters
3) n_estimators (covered before)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

n_estimator = range(50, 500, 50)

auc=[]
for nt in n_estimator:
    xgboostmodel.n_estimators = nt
    
#     xgboostmodel = XGBClassifier(n_estimators=nt,eta=0.3,min_child_weight=3,max_depth=2)
    
    cv_score = cross_val_score(xgboostmodel, data,classlabel,cv=10,scoring='roc_auc').mean()
    auc.append(cv_score)

plt.plot(n_estimator,auc)
plt.xlabel('n_estimators', fontsize=16)
plt.ylabel('AUC', fontsize=16)

#### Booster Parameters
4) min_child_weight [default=1]
    - Defines the minimum sum of weights of all observations required in a child.
    - This refers to min “sum of weights” of observations.
    - Used to control over-fitting. Higher values prevent a model from learning relations which might be highly specific to the particular sample selected for a tree.
    - Too high values can lead to under-fitting hence, it should be tuned using CV.
    
** stop trying to split once your sample size in a node goes below a given threshold.


In [ ]:
Min_child_weight = range(1,15)

auc=[]

for mcw in Min_child_weight:
    xgboostmodel.min_child_weight = mcw
#     xgboostmodel = XGBClassifier(n_estimators=75,eta=0.3,min_child_weight=mcw,max_depth=2, gamma=1.5)
    
    cv_score = cross_val_score(xgboostmodel, data,classlabel,cv=10,scoring='roc_auc').mean()
    auc.append(cv_score)

plt.plot(Min_child_weight,auc)
plt.xlabel('Min_child_weight', fontsize=16)
plt.ylabel('AUC', fontsize=16)


#### Booster Parameters
5) gamma [default=0]
    - A node is split only when the resulting split gives a positive reduction in the loss function. Gamma specifies the minimum loss reduction required to make a split.
    - Makes the algorithm conservative. The values can vary depending on the loss function and should be tuned.

** The complexity cost by introducing additional leaf

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

gamma = np.arange(0, 1, 0.01)

auc=[]
for gm in gamma:
    xgboostmodel.gamma = gm
#     xgboostmodel = XGBClassifier(n_estimators=75,eta=0.3,min_child_weight=3,max_depth=2, gamma=gm)
    
    cv_score = cross_val_score(xgboostmodel, data,classlabel,cv=10,scoring='roc_auc').mean()
    auc.append(cv_score)

plt.plot(gamma,auc)
plt.xlabel('gamma', fontsize=16)
plt.ylabel('AUC', fontsize=16)


#### Booster Parameters
6) lambda [default=1]
    - L2 regularization term on weights (analogous to Ridge regression)
    - This used to handle the regularization part of XGBoost. Though many data scientists don’t use it often, it should be explored to reduce overfitting.

In [ ]:
reg_lambda = [1e-5,1e-3, 1e-4 ,1e-2, 0.1, 0.2, 0.5, 0.9, 1,5,10,20,50, 100]

auc=[]

for regl in reg_lambda:
    xgboostmodel.reg_lambda = regl
    
#     xgboostmodel = XGBClassifier(n_estimators=75,eta=0.3,min_child_weight=5,max_depth=2, gamma=1.5, reg_lambda = regl)
    
    cv_score = cross_val_score(xgboostmodel, data,classlabel,cv=10,scoring='roc_auc').mean()
    auc.append(cv_score)

plt.plot(reg_lambda,auc)
plt.xlabel('reg_lambda', fontsize=16)
plt.ylabel('AUC', fontsize=16)


### Grid Search for Parameter Tuning

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
xgboostmodel = XGBClassifier(
    n_estimators=50,
    eta=0.3,
    min_child_weight=1,
    max_depth=3, 
    # add the following to remove two depreciation warning
    eval_metric='mlogloss', 
    use_label_encoder=False
)

# xgboostmodel = XGBClassifier(n_estimators=nt,eta=0.3,min_child_weight=3,max_depth=2)

n_estimators = [50,100,200,300]
max_depth = [2,4,6,8,10]
tuned_parameters = dict(max_depth=max_depth, n_estimators=n_estimators)

kfold = StratifiedKFold(n_splits=10, random_state=7, shuffle=True)

clf = GridSearchCV(xgboostmodel, tuned_parameters, cv=kfold, scoring='roc_auc')

clf.fit(data,classlabel)

print("Best parameters set found on development set:")
print(clf.best_params_)
